Import all required libraries

In [1]:
from die import Die
from grid import grid
from numpy import array
from numpy import ndarray

Define the backtracking function

In [2]:
def backtrack(row: int,
              col: int,
              die: Die,
              score: int,
              turn: int,
              grid: array) -> bool:
    """
    Returns true if a satisfiable path has been found.
    """
    # Remember the value of this cell
    c: int = grid[row][col]

    # Check: Is there a number on this die-face?
    index_of_top_face: int = die.index_of_top_face()
    # Remember whether the top-face was set during this function call
    top_face_number_set_here: bool = False

    if (row, col) != (5, 0):
        if not die.die_face_has_number(index_of_top_face):
            # Since c = s + t x n, n exists iff t divides c - s
            if (c - score) % turn == 0:
                die.set_die_face_number(index_of_top_face, (c - score) // turn)
                top_face_number_set_here = True
            else:
                # No integer will work as the die-face
                return False

        # Check: Given the number on the top-face, does the equation still hold?
        if c != score + turn * die.get_die_face_number(index_of_top_face):
            # We only enter this conditional if the die-face number was not set
            # in this function call, because if it was, n would have been chosen
            # by construction to satisfy the equation.
            #
            # Thus, we don't delete the die-face number here, because we don't know
            # how many cells in the path behind us relied on this die face, so we have to
            # let the function-call that was responsible for this die-face delete it.
            return False


        # Mark this square as visited
        grid[row][col] = 0

    # Check: Are we on the last square?
    if row == 0 and col == 5:
        return True

    # Check: Does rolling left work?
    die.tip_left()
    if col > 0 and grid[row][col - 1] != 0 and backtrack(row, col - 1, die, c, turn + 1, grid):
        return True
    die.tip_right()

    # Check: Does rolling right work?
    die.tip_right()
    if col + 1 < len(grid[0]) and grid[row][col + 1] != 0 and backtrack(row, col + 1, die, c, turn + 1, grid):
        return True
    die.tip_left()

    # Check: Does rolling up work?
    die.tip_up()
    if row > 0 and grid[row - 1][col] != 0 and backtrack(row - 1, col, die, c, turn + 1, grid):
        return True
    die.tip_down()

    # Check: Does rolling down work?
    die.tip_down()
    if row + 1 < len(grid) and grid[row + 1][col] != 0 and backtrack(row + 1, col, die, c, turn + 1, grid):
        return True
    die.tip_up()

    # Everything failed. If you had set a die-face here, reset it.
    if top_face_number_set_here:
        del die.die_face_numbers[index_of_top_face]

    # Reset the cell (because you set it to 0) and then report the failure
    grid[row][col] = c
    return False


Print the grid before starting just for reference

In [3]:
print(grid)

[[ 57  33 132 268 492 732]
 [ 81 123 240 443 353 508]
 [186  42 195 704 452 228]
 [ -7   2 357 452 317 395]
 [  5  23  -4 592 445 620]
 [  0  77  32 403 337 452]]


Run the backtracker

In [7]:
print("Success" if backtrack(row=5, col=0, die=Die(), score=0, turn=0, grid=grid) else "Failure")

Failure


Inspect the grid

In [5]:
print(grid)

[[ 57  33 132 268 492 732]
 [ 81 123 240 443 353 508]
 [186  42 195 704 452 228]
 [ -7   2 357 452 317 395]
 [  5  23  -4 592 445 620]
 [  0  77  32 403 337 452]]
